# <center>Analyzing Neonatal EEG Seizure Recordings</center>
## <center>November 9th, 2019</center>
Easton Potokar

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import mne                               #package to handle EEG data files
import os
import seaborn

plt.style.use("seaborn")
plt.rcParams["figure.figsize"] = [12,5]
plt.rcParams["figure.dpi"] = 200

## 1 Introduction

Neonatal seizures are a common occurrence and require immediate care. Detection is only possible through continuous electroencephalogram (EEG) monitoring. Unfortunately, this places a heavy burden on NICUs (Newborn Intensive Care Units) due to the special expertise needed to interpret EEGs that is generally not available in a NICU. Alternative options include a simplified easy-to-read trend of the EEG output known as an amplitude integrated EEG (aEEG). While it has its strengths, drawbacks include short duration and low amplitude of seizures, causing them to be missed entirely. 

Continuous multichannel EEG is the gold standard for detecting seizures but expert interpretation is not readily available to NICUs. Alternatives include providing experts remote access to the EEG, but this still requires 24 hour surveillance, also a heavy load.

The dataset that will be used is available through a public repository containing EEG recordings of 79 term neonates admitted to the NICU, with an meidan duration of 74 minutes [1]. Each EEG includes 10 channels of data, each recorded at 256Hz, and thus containing frequencies up to 128HZ. These recordings were examined by three experts with their labelings of either a seizure being present or not included at minute intervals. 

## 2 Data Scraping

The data can be found at https://zenodo.org/record/2547147, and is best downloaded using the pip package `zenodo-get`. By simply running `pip install zenodo-get` followed by `zenodo_get.py 10.5281/zenodo.2547147` downloads all data and checks the md5sums to ensure everything downloaded properly. Since this process was simple enough, no additional scraping methods were needed.

# 3 Data Cleaning

## 3.1 Converting to Pandas

The data is stored in a mixture of `.csv` and `.edf` files. The `.edf` files are a standard for EEG data, and can be read in using the python package `mne`.